In [31]:
import os
import pickle

import numpy as np
import pandas as pd

In [32]:
seed = 20170901

In [33]:
prediction_file = os.path.join('../intermediate/{}/prediction_seed{}.pkl'
                               .format(seed, seed))

with open(prediction_file, 'rb') as f:
    prediction = pickle.load(f)

In [17]:
data_df = pd.read_csv('../data/wttsf/train_1.csv', index_col='Page')
key_df = pd.read_csv('../data/wttsf/key_1.csv')
key_df['Date'] = key_df['Page'].apply(lambda a: a[-10:])
key_df['Page'] = key_df['Page'].apply(lambda a: a[:-11])

In [34]:
future_start = (pd.Timestamp('2017-01-01') - pd.Timestamp('2017-01-01')).days
future_end = (pd.Timestamp('2017-03-01') - pd.Timestamp('2017-01-01')).days
future_period = future_end + 1 - future_start

visits = np.zeros(key_df.shape[0])
for i in range(0, len(visits), future_period):
    page = key_df['Page'][i]
    page_index = data_df.index.get_loc(page)
    visits[i:(i+future_period)] = prediction[page_index,
                                             future_start:(future_end+1)]

key_df['Visits'] = visits
submission_file = os.path.join('../intermediate/{}/submission_{}.csv'
                               .format(seed, seed))
key_df[['Id', 'Visits']].to_csv(submission_file, float_format='%.0f',
                                index=False)